In [1]:
#TODO: reduce size vocabulary
#TODO: check size embedding
#TODO: tensorboard

In [2]:
import pandas as pd

df = pd.read_csv('data/crowdflower_data.csv')
df.head()

,sentiment,content
0,sadness,Layin n bed with a headache ughhhh...waitin o...
1,sadness,Funeral ceremony...gloomy friday...
2,neutral,@dannycastillo We want to trade with someone w...
3,worry,Re-pinging @ghostridah14: why didn't you go to...
4,sadness,"I should be sleep, but im not! thinking about ..."


In [3]:
df['content'].apply(lambda x: len(x.split(' '))).std()

7.236480363011788

In [4]:
num_classes = len(df['sentiment'].unique())

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=None)
tokenizer.fit_on_texts(df['content'])
sequences = tokenizer.texts_to_sequences(df['content'])
sequences = pad_sequences(sequences, maxlen=40)

/Users/arnaudstiegler/miniconda3/envs/emotion_detection/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/arnaudstiegler/miniconda3/envs/emotion_detection/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/arnaudstiegler/miniconda3/envs/emotion_detection/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16"

In [6]:
import numpy as np

emotion_dict = {}
labels = df['sentiment']

for k in range(len(labels.unique())):
    emotion_dict[labels.unique()[k]] = k
    
encoded_labels = np.array(list(labels.map(lambda x: emotion_dict[x])))

In [7]:
embeddings_index = {}
with open('data/embedding/glove.6B.100d.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [8]:
#Preparing the embedding

MAX_NUM_WORDS = 30000
EMBEDDING_DIM = 100

num_words = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


In [9]:
embedding_matrix.shape

(37339, 100)

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sequences, encoded_labels, stratify=encoded_labels)

In [11]:
import tensorflow as tf

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1024).batch(128)
test_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test)).shuffle(1024).batch(128)

In [13]:
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Conv1D, MaxPool1D,MaxPool2D, Flatten
from tensorflow.keras.initializers import Constant

model = tf.keras.Sequential()

model.add(Embedding(num_words, EMBEDDING_DIM, embeddings_initializer=Constant(embedding_matrix),
                            input_length=40,
                            trainable=False))
model.add(Dropout(rate=0.3))
model.add(LSTM(64, return_sequences=True))
model.add(Conv1D(32,10))
model.add(Conv1D(32,5))
model.add(MaxPool1D())
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 100)           3733900   
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 100)           0         
_________________________________________________________________
unified_lstm_1 (UnifiedLSTM) (None, 40, 64)            42240     
_________________________________________________________________
conv1d (Conv1D)              (None, 31, 32)            20512     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 27, 32)            5152      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 13, 32)            0         
_________________________________________________________________
flatten (Flatten)            (None, 416)              

In [14]:
import tensorflow as tf
assert tf.__version__ >= "2.0"

In [15]:
%load_ext tensorboard.notebook
!rm -rf ./tensorboard-logs/ # Clear any logs from previous runs

import datetime 
import os

date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join("./tensorboard-logs/", date)
print("Writing logs to", log_dir)
exp_dir = os.path.join(log_dir, "softmax")

tb_callback = tf.keras.callbacks.TensorBoard(log_dir=exp_dir)

Writing logs to ./tensorboard-logs/20191101-180633


In [16]:
model.fit(X_train, y_train, 
          epochs = 10, 
          validation_data=(X_test,y_test),
         callbacks=[tb_callback])

Train on 20603 samples, validate on 6868 samples
Epoch 1/10
20603/20603 [==============================] - 23s 1ms/sample - loss: 1.3195 - accuracy: 0.3805 - val_loss: 1.2845 - val_accuracy: 0.3990
Epoch 2/10
20603/20603 [==============================] - 21s 1ms/sample - loss: 1.2525 - accuracy: 0.4297 - val_loss: 1.2298 - val_accuracy: 0.4458
Epoch 3/10
20603/20603 [==============================] - 21s 1ms/sample - loss: 1.2239 - accuracy: 0.4445 - val_loss: 1.2213 - val_accuracy: 0.4461
Epoch 4/10
20603/20603 [==============================] - 21s 1ms/sample - loss: 1.2121 - accuracy: 0.4477 - val_loss: 1.2127 - val_accuracy: 0.4495
Epoch 5/10
20603/20603 [==============================] - 28s 1ms/sample - loss: 1.1969 - accuracy: 0.4559 - val_loss: 1.2078 - val_accuracy: 0.4537
Epoch 6/10
20603/20603 [==============================] - 22s 1ms/sample - loss: 1.1830 - accuracy: 0.4608 - val_loss: 1.2107 - val_accuracy: 0.4554
Epoch 7/10
20603/20603 [==============================] -

In [17]:
%tensorboard --logdir "$log_dir"